# BRW Estimation

Replication code of the BRW shock by Bu, Wu, and Rogers (2021).

In [9]:
import pandas as pd

maturity = 2
year = 2020
month = 12
monthname = "Dec"
MAR2020 = 0

## 1. Calculating the difference of the 2-year Treasury series (DGS2)

Load the data and rename columns

In [8]:
data = pd.read_excel(r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\DGS2.xls")
data.rename(columns={"observation_date": "date"}, inplace=True)
data.head()

,date,DGS2
0,1994-01-03,4.30
1,1994-01-04,4.29
2,1994-01-05,4.31
3,1994-01-06,4.24
4,1994-01-07,4.09


Convert date to datetime format and create additional date-related columns

In [10]:
data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.day
data["mdate"] = data["date"].dt.to_period("M")

Filter data for the specified period (up to September 2019 - Cut-off date used for the estimation in the research paper.)

In [17]:
data = data[data["mdate"] <= pd.Period("2019-09")]
data = data[data["mdate"] >= pd.Period("1994-01")]

Calculate the difference of the series (dgs_d)

In [18]:
data["dgs_d"] = data["DGS" + str(maturity)] - data["DGS" + str(maturity)].shift(1)
data.head()

,date,DGS2,year,month,day,mdate,dgs_d
0,1994-01-03,4.30,1994,1,3,1994-01,NaN
1,1994-01-04,4.29,1994,1,4,1994-01,-0.01
2,1994-01-05,4.31,1994,1,5,1994-01,0.02
3,1994-01-06,4.24,1994,1,6,1994-01,-0.07
4,1994-01-07,4.09,1994,1,7,1994-01,-0.15


Set "mdate" as the index and convert it to a time series

In [19]:
data.set_index("mdate", inplace=True)
data.index = pd.to_datetime(data.index.to_timestamp())

Drop unnecessary columns

In [20]:
data.drop(columns=["year", "month", "day"], inplace=True)
data.head()

,date,DGS2,dgs_d
mdate,,,
1994-01-01,1994-01-03,4.30,NaN
1994-01-01,1994-01-04,4.29,-0.01
1994-01-01,1994-01-05,4.31,0.02
1994-01-01,1994-01-06,4.24,-0.07
1994-01-01,1994-01-07,4.09,-0.15


Save the resulting DataFrame to a new file

In [21]:
data.to_pickle("temp.pkl")

## 2. Adding FOMC dates

Load the FOMC dates from the Excel file

In [31]:
fomc_dates = pd.read_excel("C:/Users/Alysson/Documents/GitHub/Monetary-Shocks/FOMCdate.xlsx")
fomc_dates.rename(columns={"date": "FOMC_date"}, inplace=True)
fomc_dates.head()

,FOMC_date,q
0,1936-03-19,1
1,1936-05-25,1
2,1936-11-20,1
3,1937-01-26,1
4,1937-03-15,1


Convert date to datetime format and create additional date-related columns

In [32]:
fomc_dates["FOMC_date"] = pd.to_datetime(fomc_dates["FOMC_date"])
fomc_dates["year"] = fomc_dates["FOMC_date"].dt.year
fomc_dates["month"] = fomc_dates["FOMC_date"].dt.month
fomc_dates["day"] = fomc_dates["FOMC_date"].dt.day
fomc_dates["mdate"] = fomc_dates["FOMC_date"].dt.to_period("M")

MAR2020 = 0

if MAR2020 == 0:
    fomc_dates = fomc_dates[~((fomc_dates["year"] == 2020) & (fomc_dates["month"] == 3) & (fomc_dates["day"] == 15))]


Drop unnecessary columns

In [33]:
fomc_dates.drop(columns=["year", "month", "day"], inplace=True)

Load the previous DataFrame "temp.dta" 

In [34]:
temp_data = pd.read_pickle("temp.pkl")

Merge the two DataFrames on the 'date' column

In [35]:
merged_data = pd.merge(temp_data, fomc_dates, how="inner", left_on="date", right_on="FOMC_date")

Drop rows where 'q' is missing (denoted as NaN)

In [36]:
merged_data = merged_data.dropna(subset=["q"])

Replace missing values (NaN) in 'q' with 0

In [42]:
merged_data["q"].fillna(0, inplace=True)
merged_data.head()

,date,DGS2,dgs_d,FOMC_date,q,mdate
0,1994-02-04,4.42,0.14,1994-02-04,1,1994-02
1,1994-02-11,4.44,-0.01,1994-02-11,0,1994-02
2,1994-03-22,5.03,-0.06,1994-03-22,1,1994-03
3,1994-03-29,5.19,0.07,1994-03-29,0,1994-03
4,1994-05-17,5.88,-0.15,1994-05-17,1,1994-05


Save the merged DataFrame to a new file

In [40]:
merged_data.to_pickle("temp.pkl")

## 3. Estimation

Load the data from "feds200628.csv" 

In [44]:
yield_data = pd.read_csv(r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\feds200628.csv")

Convert 'date' column to datetime format